## Loading data

In [2]:
pip install scikit-learn

  Using cached scikit_learn-1.3.1-cp310-cp310-win_amd64.whl (9.3 MB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
  Using cached scipy-1.11.3-cp310-cp310-win_amd64.whl (44.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from torch import nn
from helper import preprocess_df, preprocess_text
import optuna

c:\Users\ext.minh.triet.chau\code\dstest\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
data = pd.read_csv('train_data.csv')

In [ ]:
data.head(10)

,box,text,label,words,linking,id,file
0,"[394, 145, 433, 162]",ITEM:,question,"[{'box': [394, 145, 433, 162], 'text': 'ITEM:'}]","[[0, 15]]",0,71206427
1,"[109, 112, 151, 129]",DATE:,question,"[{'box': [109, 112, 151, 129], 'text': 'DATE:'}]","[[1, 13]]",1,71206427
2,"[110, 140, 159, 155]",BRAND:,question,"[{'box': [110, 140, 159, 155], 'text': 'BRAND:'}]","[[2, 14]]",2,71206427
3,"[109, 183, 168, 198]",SUMMARY,question,"[{'box': [109, 183, 168, 198], 'text': 'SUMMAR...",[],3,71206427
4,"[175, 184, 193, 195]",OF,question,"[{'box': [175, 184, 193, 195], 'text': 'OF'}]",[],4,71206427
5,"[201, 184, 267, 198]",PROJECT:,question,"[{'box': [201, 184, 267, 198], 'text': 'PROJEC...","[[5, 7], [5, 6]]",5,71206427
6,"[317, 186, 385, 199]",Attached,answer,"[{'box': [317, 186, 385, 199], 'text': 'Attach...","[[5, 6]]",6,71206427
7,"[282, 186, 310, 197]",See,answer,"[{'box': [282, 186, 310, 197], 'text': 'See'}]","[[5, 7]]",7,71206427
8,"[103, 404, 173, 418]",FUNDING:,question,"[{'box': [103, 404, 173, 418], 'text': 'FUNDIN...","[[8, 22], [8, 23]]",8,71206427
9,"[101, 559, 196, 574]",SIGNATURES:,header,"[{'box': [101, 559, 196, 574], 'text': 'SIGNAT...","[[9, 24], [9, 25], [9, 26], [9, 27], [9, 28]]",9,71206427


## Create dataset

In [ ]:
data['text'] = preprocess_text(data['text'])

In [ ]:
data.dropna(subset='text').isna().sum()

box        0
text       0
label      0
words      0
linking    0
id         0
file       0
dtype: int64

In [ ]:
data = preprocess_df(data)

In [ ]:
train_df, val_df = train_test_split(data, test_size=0.5, random_state=23)

In [ ]:
train_df

,box,text,label,words,linking,id,file
2899,"[175, 556, 321, 570]",sp sneak preview,answer,"[{'text': 'SP', 'box': [175, 556, 195, 569]}, ...","[[44, 46]]",46,00920294
203,"[22, 490, 117, 506]",fowlers oil (308 -21),answer,"[{'text': 'Fowlers', 'box': [22, 490, 60, 505]...","[[48, 61]]",61,81619511_9513
5301,"[532, 52, 670, 67]",submission date:,header,"[{'text': 'SUBMISSION', 'box': [532, 54, 624, ...","[[9, 13], [9, 14], [9, 15], [9, 16]]",9,92657311_7313
191,"[156, 419, 203, 447]",ind. lor volume,question,"[{'text': 'IND.', 'box': [156, 422, 178, 433]}...","[[49, 54], [49, 55], [49, 56], [49, 57], [49, ...",49,81619511_9513
3629,"[265, 218, 453, 236]","sample no. 1166, rd 308",answer,"[{'text': 'Sample', 'box': [265, 219, 312, 236...","[[9, 10]]",10,01197604
...,...,...,...,...,...,...,...
3048,"[141, 110, 654, 128]","diethyl 3, 3-dimethyl-2-oxo-1, 4-cyclopentaned...",answer,"[{'text': 'Diethyl', 'box': [141, 112, 207, 12...","[[0, 71]]",71,01073843
2031,"[52, 364, 131, 381]",projected:,header,"[{'box': [52, 364, 131, 381], 'text': 'Project...","[[2, 44], [2, 45], [2, 46], [2, 47]]",2,0012529295
1080,"[81, 168, 245, 183]",(4) product length:,question,"[{'text': '(4)', 'box': [81, 168, 106, 183]}, ...","[[26, 4], [22, 26]]",26,0060308461
4952,"[80, 873, 194, 886]",xer 264/ rev 5/ 68,other,"[{'text': 'XER', 'box': [80, 873, 102, 886]}, ...",[],28,0011859695


In [ ]:
data

,box,text,label,words,linking,id,file
0,"[394, 145, 433, 162]",item:,question,"[{'box': [394, 145, 433, 162], 'text': 'ITEM:'}]","[[0, 15]]",0,71206427
1,"[109, 112, 151, 129]",date:,question,"[{'box': [109, 112, 151, 129], 'text': 'DATE:'}]","[[1, 13]]",1,71206427
2,"[110, 140, 159, 155]",brand:,question,"[{'box': [110, 140, 159, 155], 'text': 'BRAND:'}]","[[2, 14]]",2,71206427
3,"[109, 183, 168, 198]",summary,question,"[{'box': [109, 183, 168, 198], 'text': 'SUMMAR...",[],3,71206427
4,"[175, 184, 193, 195]",of,question,"[{'box': [175, 184, 193, 195], 'text': 'OF'}]",[],4,71206427
...,...,...,...,...,...,...,...
7406,"[397, 881, 528, 914]",topline w/o 12 /18 /95 final report 12/ 31/ 95,answer,"[{'text': 'Topline', 'box': [398, 883, 443, 89...","[[59, 60]]",60,80310840a
7407,"[577, 742, 635, 759]",$100 600,other,"[{'text': '$100', 'box': [577, 742, 606, 757]}...",[],61,80310840a
7408,"[398, 761, 560, 779]",revised costs (if any),question,"[{'text': '', 'box': [398, 761, 409, 774]}, {'...",[],62,80310840a
7409,"[397, 788, 532, 806]",fieldwork schedule:,question,"[{'text': 'Fieldwork', 'box': [397, 788, 464, ...","[[63, 58]]",63,80310840a


In [ ]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    """
    This is a small dataset, therefore I save everything in memory
    """
    def __init__(self, tokenizer, texts, labels):
        assert len(texts) == len(labels)
        self.index_label = dict(zip(range(4), ['answer', 'header', 'other', 'question']))
        self.label_index = dict(zip(['answer', 'header', 'other', 'question'], range(4)))
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.input_ids = []
        self.attn_masks = []
        for text in self.texts:
            _ = tokenizer(text, truncation=True)
            input_id, attn_mask = _['input_ids'], _['attention_mask']
            self.input_ids.append(input_id)
            self.attn_masks.append(attn_mask)
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {#"text": self.tokenizer.tokenize(self.texts[idx]), 
        "label": self.label_index[self.labels[idx]], 
        'input_ids': self.input_ids[idx], 
        'attention_mask': self.attn_masks[idx]}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
train_dataset = TextDataset(tokenizer, train_df['text'].values, train_df['label'].values)
val_dataset = TextDataset(tokenizer, val_df['text'].values, val_df['label'].values)

In [ ]:
assert len(train_dataset)+len(val_dataset)==len(data)

## Define loss function
Since `other` and `header` are under-represented, we will add more weight to them

In [ ]:
index_label = dict(zip(range(4), ['answer', 'header', 'other', 'question']))
label_index = dict(zip(['answer', 'header', 'other', 'question'], range(4)))

In [ ]:
# todo whole dataset count label, inverse porpotional, normalize

In [ ]:
count_labels_dict = data['label'].value_counts().to_dict()

In [ ]:
count_labels_dict['answer']

2721

In [ ]:
0.0022675736961451248/0.0003074085459575776

7.376417233560091

In [ ]:
print("Before", count_labels_dict)
for key in count_labels_dict.keys():
    # inverse
    count_labels_dict[key] = 1/count_labels_dict[key]
print("After", count_labels_dict)

Before {'question': 3253, 'answer': 2721, 'other': 833, 'header': 441}
After {'question': 0.0003074085459575776, 'answer': 0.0003675119441381845, 'other': 0.0012004801920768306, 'header': 0.0022675736961451248}


In [ ]:
weight_tensor = torch.Tensor([count_labels_dict['answer'],count_labels_dict['header'],
                              count_labels_dict['other'],count_labels_dict['question']])
weight_tensor = torch.Tensor([1,1.1,1.1,1])

In [ ]:
class WeightedTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = nn.CrossEntropyLoss(weight=weight_tensor)
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

## Start training

### Hyperparams tuning

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [ ]:
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4, id2label=index_label, label2id=label_index,
    problem_type='single_label_classification'
    )

In [ ]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-3, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 1e-3, 1e-1, log=True),
    }

In [ ]:
torch.manual_seed(2)

training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    load_best_model_at_end=False,
    report_to="tensorboard",
    disable_tqdm=False
)

trainer = WeightedTrainer(
    model=None,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    model_init=model_init
)

loading configuration file config.json from cache at /Users/chaum/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "answer",
    "1": "header",
    "2": "other",
    "3": "question"
  },
  "initializer_range": 0.02,
  "label2id": {
    "answer": 0,
    "header": 1,
    "other": 2,
    "question": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.0",
  "vocab_size": 30522
}

loading weights file pytor

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="minimize",
    backend="optuna",
    timeout=1000,
    hp_space=hp_space
)

[I 2023-02-12 17:22:24,692] A new study created in memory with name: no-name-147a096c-402b-40ce-9eac-356a12d73e33
Trial: {'learning_rate': 0.00016176775440442069, 'per_device_train_batch_size': 16, 'weight_decay': 0.025364953529501567}
loading configuration file config.json from cache at /Users/chaum/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "answer",
    "1": "header",
    "2": "other",
    "3": "question"
  },
  "initializer_range": 0.02,
  "label2id": {
    "answer": 0,
    "header": 1,
    "other": 2,
    "question": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "probl

Epoch,Training Loss,Validation Loss
1,No log,0.648360
2,No log,0.670285
3,0.614300,0.741448


***** Running Evaluation *****
  Num examples = 3624
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3624
  Batch size = 8
Saving model checkpoint to models/run-0/checkpoint-500
Configuration saved in models/run-0/checkpoint-500/config.json
Model weights saved in models/run-0/checkpoint-500/pytorch_model.bin
tokenizer config file saved in models/run-0/checkpoint-500/tokenizer_config.json
Special tokens file saved in models/run-0/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3624
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


[I 2023-02-12 20:24:38,478] Trial 0 finished with value: 0.7414484620094299 and parameters: {'learning_rate': 0.00016176775440442069, 'per_device_train_batch_size': 16, 'weight_decay': 0.025364953529501567}. Best is trial 0 with value: 0.7414484620094299.


In [ ]:
best_trial.hyperparameters

{'learning_rate': 0.00016176775440442069,
 'per_device_train_batch_size': 16,
 'weight_decay': 0.025364953529501567}

### Train with best hyperparams in the whole dataset

In [ ]:
whole_dataset = TextDataset(tokenizer, data['text'].values, data['label'].values)

```
{'learning_rate': 0.0008549921087552112,
 'per_device_train_batch_size': 32,
 'weight_decay': 0.08477673461329668}
```
With these hyperparams, we will train with all the data to predict the test set

In [ ]:
# some values below are from previous hyperpams search
best_training_args = TrainingArguments(
    learning_rate=best_trial.hyperparameters['learning_rate'],  # 0.00022191984360513044
    per_device_train_batch_size=best_trial.hyperparameters['per_device_train_batch_size'],   # 16
    weight_decay=best_trial.hyperparameters['weight_decay'],    # 0.004611087956707392
    num_train_epochs=15,
    output_dir="models",
    per_device_eval_batch_size=16,
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    evaluation_strategy="steps",    
    save_total_limit = 3,
    report_to="tensorboard",
    disable_tqdm=False,
    load_best_model_at_end=True
)

PyTorch: setting up devices


In [ ]:
best_model = model_init(None)
best_trainer = WeightedTrainer(
    model=best_model,
    args=best_training_args,
    train_dataset=whole_dataset,  # not train_dataset anymore, as we will train on the whole data
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

loading configuration file config.json from cache at /Users/chaum/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "answer",
    "1": "header",
    "2": "other",
    "3": "question"
  },
  "initializer_range": 0.02,
  "label2id": {
    "answer": 0,
    "header": 1,
    "other": 2,
    "question": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.0",
  "vocab_size": 30522
}

loading weights file pytor

In [ ]:
best_trainer.train()

/Users/chaum/code/dstest/venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7248
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6795
  Number of trainable parameters = 66956548


Step,Training Loss,Validation Loss
200,No log,0.677450
400,No log,0.609645
600,0.771500,0.578329
800,0.771500,0.475208
1000,0.568600,0.429468
1200,0.568600,0.377185
1400,0.568600,0.345363
1600,0.434300,0.310338
1800,0.434300,0.311550
2000,0.354600,0.255763


***** Running Evaluation *****
  Num examples = 3624
  Batch size = 16
Saving model checkpoint to models/checkpoint-200
Configuration saved in models/checkpoint-200/config.json
Model weights saved in models/checkpoint-200/pytorch_model.bin
tokenizer config file saved in models/checkpoint-200/tokenizer_config.json
Special tokens file saved in models/checkpoint-200/special_tokens_map.json
Deleting older checkpoint [models/checkpoint-1400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3624
  Batch size = 16
Saving model checkpoint to models/checkpoint-400
Configuration saved in models/checkpoint-400/config.json
Model weights saved in models/checkpoint-400/pytorch_model.bin
tokenizer config file saved in models/checkpoint-400/tokenizer_config.json
Special tokens file saved in models/checkpoint-400/special_tokens_map.json
Deleting older checkpoint [models/checkpoint-1600] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3624
  Batch

Model weights saved in models/checkpoint-3600/pytorch_model.bin
tokenizer config file saved in models/checkpoint-3600/tokenizer_config.json
Special tokens file saved in models/checkpoint-3600/special_tokens_map.json
Deleting older checkpoint [models/checkpoint-3000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3624
  Batch size = 16
Saving model checkpoint to models/checkpoint-3800
Configuration saved in models/checkpoint-3800/config.json
Model weights saved in models/checkpoint-3800/pytorch_model.bin
tokenizer config file saved in models/checkpoint-3800/tokenizer_config.json
Special tokens file saved in models/checkpoint-3800/special_tokens_map.json
Deleting older checkpoint [models/checkpoint-3200] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3624
  Batch size = 16
Saving model checkpoint to models/checkpoint-4000
Configuration saved in models/checkpoint-4000/config.json
Model weights saved in models/checkpoint-4000/pyto

TrainOutput(global_step=6795, training_loss=0.2902216354830463, metrics={'train_runtime': 2716.1069, 'train_samples_per_second': 40.028, 'train_steps_per_second': 2.502, 'total_flos': 620772649722624.0, 'train_loss': 0.2902216354830463, 'epoch': 15.0})

In [ ]:
best_trainer.save_model("models/best")

Saving model checkpoint to models/best
Configuration saved in models/best/config.json
Model weights saved in models/best/pytorch_model.bin
tokenizer config file saved in models/best/tokenizer_config.json
Special tokens file saved in models/best/special_tokens_map.json
